In [3]:
import numpy as np
import nibabel as nib
from dipy.core.gradients import gradient_table
from dipy.reconst.csdeconv import auto_response
from dipy.reconst.csdeconv import recursive_response, response_from_mask

In [4]:
# load all the data
#load atlas data
mylabel = 118
mypath='data/'
labels_img = nib.load(mypath+'fa_labels_warp_N54917_RAS_332.nii.gz')
labels = labels_img.get_data()
print 'loading label data finished'


#load 4D image data
img = nib.load(mypath+'N54917_nii4D_RAS.nii')
data = img.get_data()
affine = img.affine
print 'loading img data finished'

#load bvals and bvecs data
bvals = np.loadtxt(mypath+'N54917_RAS_ecc_bvals.txt')
bvecs = np.loadtxt(mypath+'N54917_RAS_ecc_bvecs.txt')
gtab = gradient_table(bvals, bvecs)
print 'gradient table calculation finished'

#Build Brain Mask
bm = np.where(labels==0, False, True)
mask = bm
print 'masking the brain finished'

# white matter mask
logic1 = np.logical_and(labels>117, labels<148)
logic2 = np.logical_and(labels>283, labels<314)
logic = np.logical_or(logic1, logic2)
logic_ = np.logical_or(labels==150, labels==316)
wm = np.where(logic, True, np.where(logic_, True, False))

loading label data finished
loading img data finished
gradient table calculation finished
masking the brain finished


#### auto_respense

In [36]:
#auto_response with roi_radius=10
response_a1, ratio_a1, nvoxel_a1 = auto_response(gtab, data, 
                                    roi_radius=10, 
                                    fa_thr=0.7, 
                                    roi_center=None, 
                                    return_number_of_voxels=True)

print('the response function is\n{}\n'.format(response_a1)
      +'the ratio is {}\n'.format(ratio_a1)
      +'the number of voxel is {}'.format(nvoxel_a1))

the response function is
(array([4.01206489e-04, 9.41710590e-05, 9.41710590e-05]), 5738.5337)
the ratio is 0.234719680709
the number of voxel is 26


In [39]:
#auto_response with roi_radius=20
response_a2, ratio_a2, nvoxel_a2 = auto_response(gtab, data, 
                                    roi_radius=20, 
                                    fa_thr=0.7, 
                                    roi_center=None, 
                                    return_number_of_voxels=True)

print('the response function is\n{}\n'.format(response_a2)
      +'the ratio is {}\n'.format(ratio_a2)
      +'the number of voxel is {}'.format(nvoxel_a2))

the response function is
(array([0.00042301, 0.00010075, 0.00010075]), 5729.6885)
the ratio is 0.238186218094
the number of voxel is 957


In [40]:
#auto_response with roi_radius=30
response_a3, ratio_a3, nvoxel_a3 = auto_response(gtab, data, 
                                    roi_radius=30, 
                                    fa_thr=0.7, 
                                    roi_center=None, 
                                    return_number_of_voxels=True)

print('the response function is\n{}\n'.format(response_a3)
      +'the ratio is {}\n'.format(ratio_a3)
      +'the number of voxel is {}'.format(nvoxel_a3))

the response function is
(array([0.00041355, 0.00010038, 0.00010038]), 5833.895)
the ratio is 0.242722902717
the number of voxel is 3584


#### recursive_response

In [ ]:
response_r = recursive_response(gtab, data, mask=wm, sh_order=8,
                              peak_thr=0.001, init_fa=0.08,
                              init_trace=0.0021, iter=8, convergence=0.001,
                              parallel=True)
print('the recursive response function is\n{}\n'.format(response_r))

In [56]:
response_r.dwi_response

array([ 6874.94631084, -2620.88363841,   876.34702927,  -246.43869606,
          55.64382723])

#### response_function from mask

In [5]:
#response function from white matter mask
from dipy.reconst.csdeconv import response_from_mask
response_m, ratio_m = response_from_mask(gtab, data, mask=wm)
print('the response function from wm is\n{}\n'.format(response_m)
      +'the ratio is {}\n'.format(ratio_m))

the response function from wm is
(array([0.00040757, 0.00020827, 0.00020827]), 6070.5767)
the ratio is 0.510988101817



In [10]:
#response function from each region in white matter mask
from dipy.reconst.csdeconv import response_from_mask
for i in range(118, 148):
    response_m, ratio_m = response_from_mask(gtab, data, mask=labels==i)
    print('the response function from region-{} is\n{}\n'.format(i, response_m)
          +'the ratio is {}\n'.format(ratio_m))

the response function from region-118 is
(array([0.00043809, 0.00016432, 0.00016432]), 6270.5786)
the ratio is 0.375081383311

the response function from region-119 is
(array([0.00051548, 0.00019263, 0.00019263]), 6123.75)
the ratio is 0.373694683897

the response function from region-120 is
(array([0.00039085, 0.00017255, 0.00017255]), 7134.4673)
the ratio is 0.441480587862

the response function from region-121 is
(array([0.00037472, 0.000203  , 0.000203  ]), 6693.799)
the ratio is 0.541756183235

the response function from region-122 is
(array([0.00036748, 0.00018449, 0.00018449]), 6789.478)
the ratio is 0.502029965602

the response function from region-123 is
(array([0.00038209, 0.00015495, 0.00015495]), 6984.9365)
the ratio is 0.405524398228

the response function from region-124 is
(array([0.00035595, 0.00021202, 0.00021202]), 6458.5215)
the ratio is 0.595650616025

the response function from region-125 is
(array([0.00042735, 0.00023138, 0.00023138]), 6133.1313)
the ratio is 0.54